# Transaction Data Simulator

For confidentiality reasons, it is hard to find data related to fraud detection problems, despite its importance. Therefore, I will use a transaction data simulator to generate data for this project. For more details about the simulator, please refer to a [recent book about ML for fraud detection](https://fraud-detection-handbook.github.io/fraud-detection-handbook/Foreword.html). In the following, I will briefly explain the main ideas behind the transactions data generating process.